# Importing packages

In [1]:
import json
import codecs
import math
import numpy as np
import utils
import tensorflow as tf
import matplotlib.pyplot as plt

c:\users\aritra banerjee\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\aritra banerjee\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\aritra banerjee\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\aritra banerjee\envs\dl\lib\site-packages\tensorflow\python\framework\dty

# Reading data

In [2]:
# Read data
batch_size = 128
seq_max_len = 32
nb_sentiment_label = 3
embedding_size = 100
nb_linear_inside = 256
nb_lstm_inside = 256
layers = 1
TRAINING_ITERATIONS = 15000
LEARNING_RATE = 0.1
WEIGHT_DECAY = 0.0005
negative_weight = 2.0
positive_weight = 1.0
neutral_weight = 3.0

label_dict = {
    'aspositive' : 1,
    'asneutral' : 0,
    'asnegative': 2
}

data_dir = '../data/ABSA_SemEval2014/'
domain = 'Restaurants'
flag_word2vec = True
flag_addition_corpus = False
flag_change_file_structure = True
flag_train = True
flag_test = True
flag_use_sentiment_embedding = False

In [4]:
train_data, train_mask, train_binary_mask, train_label, train_seq_len, train_sentiment_for_word, test_data, test_mask, test_binary_mask, test_label, test_seq_len, test_sentiment_for_word, word_dict, word_dict_rev, embedding, aspect_list  = utils.load_data(
    domain,
    data_dir,
    flag_word2vec,
    label_dict,
    seq_max_len,
    flag_addition_corpus,
    flag_change_file_structure,
    negative_weight,
    positive_weight,
    neutral_weight,
    flag_use_sentiment_embedding
)

pos: 1431
neu: 474
neg: 539
len of train data is 2021
len of test data is 606


In [11]:
nb_sample_train = len(train_data)

# Modeling

In [9]:
graph = tf.Graph()
with graph.as_default(), tf.device('/cpu:0'):
    tf_X_train = tf.placeholder(tf.float32, shape=[None, seq_max_len, embedding_size])
    tf_X_train_mask = tf.placeholder(tf.float32, shape=[None, seq_max_len])
    tf_X_binary_mask = tf.placeholder(tf.float32, shape=[None, seq_max_len])
    tf_y_train = tf.placeholder(tf.int64, shape=[None, seq_max_len])
    keep_prob = tf.placeholder(tf.float32)
    
    ln_w = tf.Variable(tf.truncated_normal([embedding_size, nb_linear_inside], stddev=1.0 / math.sqrt(embedding_size)))
    ln_b = tf.Variable(tf.zeros([nb_linear_inside]))
    
    
    sent_w = tf.Variable(tf.truncated_normal([nb_lstm_inside, nb_sentiment_label],
                                             stddev=1.0 / math.sqrt(2 * nb_lstm_inside)))
    sent_b = tf.Variable(tf.zeros([nb_sentiment_label]))
    

    y_labels = tf.one_hot(tf_y_train, nb_sentiment_label,
                          on_value = 1.0,
                          off_value = 0.0,
                          axis = -1)
     

    X_train = tf.transpose(tf_X_train, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    X_train = tf.reshape(X_train, [-1, embedding_size])
    X_train = tf.add(tf.matmul(X_train, ln_w), ln_b)
    X_train = tf.nn.relu(X_train)
    X_train = tf.split(axis=0, num_or_size_splits=seq_max_len, value=X_train)
    
    # bidirection lstm
    # Creating the forward and backwards cells
    # X_train = tf.stack(X_train)
    # print(X_train.get_shape())
    lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(nb_lstm_inside, forget_bias=1.0)
    lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(nb_lstm_inside, forget_bias=1.0)
    # Pass lstm_fw_cell / lstm_bw_cell directly to tf.nn.bidrectional_rnn
    # if only a single layer is needed
    lstm_fw_multicell = tf.nn.rnn_cell.MultiRNNCell([lstm_fw_cell]*layers)
    lstm_bw_multicell = tf.nn.rnn_cell.MultiRNNCell([lstm_bw_cell]*layers)
    # Get lstm cell output
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_multicell,
                                                 lstm_bw_multicell,
                                                 X_train, dtype='float32')
    # output_fw, output_bw = outputs
    # outputs = tf.multiply(output_fw, output_bw)
    # outputs = tf.concat(outputs, 2)
    output_fw, output_bw = tf.split(outputs, [nb_lstm_inside, nb_lstm_inside], 2)
    sentiment = tf.reshape(tf.add(output_fw, output_bw), [-1, nb_lstm_inside]) 
    # sentiment = tf.multiply(sentiment, tf_X_train_mask)
    # sentiment = tf.reduce_mean(sentiment, reduction_indices=1)
    # sentiment = outputs[-1]
    sentiment = tf.nn.dropout(sentiment, keep_prob)
    sentiment = tf.add(tf.matmul(sentiment, sent_w), sent_b)
    sentiment = tf.split(axis=0, num_or_size_splits=seq_max_len, value=sentiment)

    # change back dimension to [batch_size, n_step, n_input]
    sentiment = tf.stack(sentiment)
    sentiment = tf.transpose(sentiment, [1, 0, 2])
    sentiment = tf.multiply(sentiment, tf.expand_dims(tf_X_binary_mask, 2))

    cross_entropy = tf.reduce_mean(tf.multiply(tf.nn.softmax_cross_entropy_with_logits(logits=sentiment, labels=y_labels), tf_X_train_mask))
    prediction = tf.argmax(tf.nn.softmax(sentiment), 2)
    correct_prediction = tf.reduce_sum(tf.multiply(tf.cast(tf.equal(prediction, tf_y_train), tf.float32), tf_X_binary_mask))
    # TODO here, fix tf_X_train_mask to 0, 1 vector
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(LEARNING_RATE, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, global_step=global_step)
    
    saver = tf.train.Saver()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instruc

# Training

In [12]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=False)) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    x_test = list()
    for i in range(len(test_data)):
        sentence = list()
        for word_id in test_data[i]:
            sentence.append(embedding[word_id])
        x_test.append(sentence)
    

    x_train = list()
    for i in range(len(train_data)):
        sentence = list()
        for word_id in train_data[i]:
            sentence.append(embedding[word_id])
        x_train.append(sentence)


    if (flag_train):
        loss_list = list()
        accuracy_list = list()

        for it in range(TRAINING_ITERATIONS):
            #generate batch (x_train, y_train, seq_lengths_train)
            if (it * batch_size % nb_sample_train + batch_size < nb_sample_train):
                index = it * batch_size % nb_sample_train
            else:
                index = nb_sample_train - batch_size

            _, correct_prediction_train, cost_train = sess.run([optimizer, correct_prediction, cross_entropy], 
                                                      feed_dict={tf_X_train: np.asarray(x_train[index : index + batch_size]),
                                                                 tf_X_train_mask: np.asarray(train_mask[index : index + batch_size]),
                                                                 tf_X_binary_mask: np.asarray(train_binary_mask[index : index + batch_size]),
                                                                 tf_y_train: np.asarray(train_label[index : index + batch_size]),
                                                                 keep_prob: 1.0})

            print('training_accuracy => %.3f, cost value => %.5f for step %d, learning_rate => %.5f' % \
                (float(correct_prediction_train)/np.sum(np.asarray(train_binary_mask[index : index + batch_size])), cost_train, it, learning_rate.eval()))
            
            loss_list.append(cost_train)
            accuracy_list.append(float(correct_prediction_train)/np.sum(np.asarray(train_binary_mask[index : index + batch_size])))

            if it % 50 == 0:
                correct_prediction_test, cost_test = sess.run([correct_prediction, cross_entropy], 
                                                  feed_dict={tf_X_train: np.asarray(x_test),
                                                             tf_X_train_mask: np.asarray(test_mask),
                                                             tf_X_binary_mask: np.asarray(test_binary_mask),
                                                             tf_y_train: np.asarray(test_label),
                                                             keep_prob: 1.0})

                print('test accuracy => %.3f , cost value  => %.5f' %(float(correct_prediction_test)/np.sum(test_binary_mask), cost_test))

                plt.plot(accuracy_list)
                axes = plt.gca()
                axes.set_ylim([0, 1.2])
                plt.title('batch train accuracy')
                plt.ylabel('accuracy')
                plt.xlabel('step')
                plt.savefig('accuracy.png')
                plt.close()

                plt.plot(loss_list)
                plt.title('batch train loss')
                plt.ylabel('loss')
                plt.xlabel('step')
                plt.savefig('loss.png')
                plt.close()

        saver.save(sess, '../ckpt/se-apect-v0.ckpt')
    else:
        saver.restore(sess, '../ckpt/se-apect-v0.ckpt')

    correct_prediction_test, prediction_test = sess.run([correct_prediction, prediction], 
                                              feed_dict={tf_X_train: np.asarray(x_test),
                                                         tf_X_train_mask: np.asarray(test_mask),
                                                         tf_X_binary_mask: np.asarray(test_binary_mask),
                                                         tf_y_train: np.asarray(test_label),
                                                         keep_prob: 1.0})


    print('test accuracy => %.3f' %(float(correct_prediction_test)/np.sum(test_binary_mask)))

training_accuracy => 0.669, cost value => 0.05191 for step 0, learning_rate => 0.10000
test accuracy => 0.657 , cost value  => 0.04503
training_accuracy => 0.602, cost value => 0.04929 for step 1, learning_rate => 0.10000
training_accuracy => 0.530, cost value => 0.05237 for step 2, learning_rate => 0.10000
training_accuracy => 0.693, cost value => 0.04963 for step 3, learning_rate => 0.10000
training_accuracy => 0.550, cost value => 0.05241 for step 4, learning_rate => 0.10000
training_accuracy => 0.586, cost value => 0.05056 for step 5, learning_rate => 0.10000
training_accuracy => 0.470, cost value => 0.06382 for step 6, learning_rate => 0.10000
training_accuracy => 0.610, cost value => 0.05240 for step 7, learning_rate => 0.10000
training_accuracy => 0.536, cost value => 0.05613 for step 8, learning_rate => 0.10000
training_accuracy => 0.490, cost value => 0.04863 for step 9, learning_rate => 0.10000
training_accuracy => 0.546, cost value => 0.05270 for step 10, learning_rate => 0.

training_accuracy => 0.149, cost value => 0.04824 for step 93, learning_rate => 0.10000
training_accuracy => 0.350, cost value => 0.05959 for step 94, learning_rate => 0.10000
training_accuracy => 0.137, cost value => 0.04887 for step 95, learning_rate => 0.10000
training_accuracy => 0.299, cost value => 0.05313 for step 96, learning_rate => 0.10000
training_accuracy => 0.142, cost value => 0.04943 for step 97, learning_rate => 0.10000
training_accuracy => 0.185, cost value => 0.04897 for step 98, learning_rate => 0.10000
training_accuracy => 0.195, cost value => 0.05054 for step 99, learning_rate => 0.10000
training_accuracy => 0.271, cost value => 0.05715 for step 100, learning_rate => 0.10000
test accuracy => 0.149 , cost value  => 0.04506
training_accuracy => 0.261, cost value => 0.06245 for step 101, learning_rate => 0.10000
training_accuracy => 0.197, cost value => 0.05286 for step 102, learning_rate => 0.10000
training_accuracy => 0.195, cost value => 0.04947 for step 103, learn

training_accuracy => 0.193, cost value => 0.04813 for step 185, learning_rate => 0.10000
training_accuracy => 0.156, cost value => 0.04868 for step 186, learning_rate => 0.10000
training_accuracy => 0.231, cost value => 0.05407 for step 187, learning_rate => 0.10000
training_accuracy => 0.117, cost value => 0.04777 for step 188, learning_rate => 0.10000
training_accuracy => 0.317, cost value => 0.05905 for step 189, learning_rate => 0.10000
training_accuracy => 0.152, cost value => 0.05024 for step 190, learning_rate => 0.10000
training_accuracy => 0.255, cost value => 0.04992 for step 191, learning_rate => 0.10000
training_accuracy => 0.180, cost value => 0.04956 for step 192, learning_rate => 0.10000
training_accuracy => 0.136, cost value => 0.04619 for step 193, learning_rate => 0.10000
training_accuracy => 0.192, cost value => 0.05619 for step 194, learning_rate => 0.10000
training_accuracy => 0.268, cost value => 0.05736 for step 195, learning_rate => 0.10000
training_accuracy => 

training_accuracy => 0.184, cost value => 0.05393 for step 276, learning_rate => 0.10000
training_accuracy => 0.265, cost value => 0.05398 for step 277, learning_rate => 0.10000
training_accuracy => 0.164, cost value => 0.04794 for step 278, learning_rate => 0.10000
training_accuracy => 0.176, cost value => 0.06134 for step 279, learning_rate => 0.10000
training_accuracy => 0.193, cost value => 0.04839 for step 280, learning_rate => 0.10000
training_accuracy => 0.174, cost value => 0.05018 for step 281, learning_rate => 0.10000
training_accuracy => 0.198, cost value => 0.05458 for step 282, learning_rate => 0.10000
training_accuracy => 0.185, cost value => 0.04959 for step 283, learning_rate => 0.10000
training_accuracy => 0.317, cost value => 0.05877 for step 284, learning_rate => 0.10000
training_accuracy => 0.185, cost value => 0.04971 for step 285, learning_rate => 0.10000
training_accuracy => 0.265, cost value => 0.05335 for step 286, learning_rate => 0.10000
training_accuracy => 

training_accuracy => 0.160, cost value => 0.05081 for step 367, learning_rate => 0.10000
training_accuracy => 0.230, cost value => 0.05420 for step 368, learning_rate => 0.10000
training_accuracy => 0.236, cost value => 0.05896 for step 369, learning_rate => 0.10000
training_accuracy => 0.232, cost value => 0.05472 for step 370, learning_rate => 0.10000
training_accuracy => 0.173, cost value => 0.05353 for step 371, learning_rate => 0.10000
training_accuracy => 0.263, cost value => 0.04790 for step 372, learning_rate => 0.10000
training_accuracy => 0.195, cost value => 0.05494 for step 373, learning_rate => 0.10000
training_accuracy => 0.163, cost value => 0.05723 for step 374, learning_rate => 0.10000
training_accuracy => 0.179, cost value => 0.04886 for step 375, learning_rate => 0.10000
training_accuracy => 0.218, cost value => 0.05197 for step 376, learning_rate => 0.10000
training_accuracy => 0.155, cost value => 0.04883 for step 377, learning_rate => 0.10000
training_accuracy => 

...

training_accuracy => 0.398, cost value => 0.05853 for step 13594, learning_rate => 0.00037
training_accuracy => 0.432, cost value => 0.04872 for step 13595, learning_rate => 0.00037
training_accuracy => 0.427, cost value => 0.04888 for step 13596, learning_rate => 0.00037
training_accuracy => 0.344, cost value => 0.05054 for step 13597, learning_rate => 0.00037
training_accuracy => 0.345, cost value => 0.04829 for step 13598, learning_rate => 0.00037
training_accuracy => 0.331, cost value => 0.05618 for step 13599, learning_rate => 0.00037
training_accuracy => 0.358, cost value => 0.05547 for step 13600, learning_rate => 0.00037
test accuracy => 0.375 , cost value  => 0.04473
training_accuracy => 0.448, cost value => 0.05436 for step 13601, learning_rate => 0.00037
training_accuracy => 0.346, cost value => 0.05465 for step 13602, learning_rate => 0.00037
training_accuracy => 0.327, cost value => 0.05293 for step 13603, learning_rate => 0.00037
training_accuracy => 0.343, cost value => 

training_accuracy => 0.339, cost value => 0.05114 for step 13683, learning_rate => 0.00037
training_accuracy => 0.307, cost value => 0.06139 for step 13684, learning_rate => 0.00037
training_accuracy => 0.425, cost value => 0.04735 for step 13685, learning_rate => 0.00037
training_accuracy => 0.367, cost value => 0.05162 for step 13686, learning_rate => 0.00037
training_accuracy => 0.407, cost value => 0.05140 for step 13687, learning_rate => 0.00037
training_accuracy => 0.407, cost value => 0.05430 for step 13688, learning_rate => 0.00037
training_accuracy => 0.398, cost value => 0.05853 for step 13689, learning_rate => 0.00037
training_accuracy => 0.441, cost value => 0.04748 for step 13690, learning_rate => 0.00037
training_accuracy => 0.412, cost value => 0.05397 for step 13691, learning_rate => 0.00037
training_accuracy => 0.350, cost value => 0.04816 for step 13692, learning_rate => 0.00037
training_accuracy => 0.331, cost value => 0.04976 for step 13693, learning_rate => 0.00037

training_accuracy => 0.308, cost value => 0.05109 for step 13772, learning_rate => 0.00037
training_accuracy => 0.339, cost value => 0.05135 for step 13773, learning_rate => 0.00037
training_accuracy => 0.344, cost value => 0.06121 for step 13774, learning_rate => 0.00037
training_accuracy => 0.466, cost value => 0.05335 for step 13775, learning_rate => 0.00037
training_accuracy => 0.328, cost value => 0.05341 for step 13776, learning_rate => 0.00037
training_accuracy => 0.353, cost value => 0.04788 for step 13777, learning_rate => 0.00037
training_accuracy => 0.333, cost value => 0.05394 for step 13778, learning_rate => 0.00037
training_accuracy => 0.283, cost value => 0.05787 for step 13779, learning_rate => 0.00037
training_accuracy => 0.425, cost value => 0.04813 for step 13780, learning_rate => 0.00037
training_accuracy => 0.387, cost value => 0.05445 for step 13781, learning_rate => 0.00037
training_accuracy => 0.431, cost value => 0.04589 for step 13782, learning_rate => 0.00037

training_accuracy => 0.412, cost value => 0.04701 for step 13861, learning_rate => 0.00037
training_accuracy => 0.398, cost value => 0.05853 for step 13862, learning_rate => 0.00037
training_accuracy => 0.435, cost value => 0.04670 for step 13863, learning_rate => 0.00037
training_accuracy => 0.487, cost value => 0.05210 for step 13864, learning_rate => 0.00037
training_accuracy => 0.328, cost value => 0.05073 for step 13865, learning_rate => 0.00037
training_accuracy => 0.344, cost value => 0.04956 for step 13866, learning_rate => 0.00037
training_accuracy => 0.350, cost value => 0.05035 for step 13867, learning_rate => 0.00037
training_accuracy => 0.360, cost value => 0.05417 for step 13868, learning_rate => 0.00037
training_accuracy => 0.391, cost value => 0.06025 for step 13869, learning_rate => 0.00037
training_accuracy => 0.392, cost value => 0.05281 for step 13870, learning_rate => 0.00037
training_accuracy => 0.333, cost value => 0.05071 for step 13871, learning_rate => 0.00037

KeyboardInterrupt: 

# Rough

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  1,
  0,
  0,